In [138]:
from pywgraph import *

In [139]:
grafo = WeightedDirectedGraph.from_dict(
    {
        "A": {"B": 1, "F": 1},
        "B": {"C": 1, "A":1},
        "C": {"A":1, "D":1},
        "D": {"A":1},
        "E": {"B":1},
        "F":{"E":1}
    }
)

In [140]:
class PathExplorer(list[str]):

    def __init__(self, path: list[str] = [], visited: set[str] = set()):
        self._path = path
        self._visited = visited

    @property
    def path(self) -> list[str]:
        return self._path
    
    @property
    def visited(self) -> set[str]:
        return self._visited
    
    def __hash__(self) -> int:
        return hash((tuple(self.path), tuple(self.visited)))
    
    def __eq__(self, other: object) -> bool:
        if isinstance(other, PathExplorer):
            return (self.path, self.visited) == (other.path, other.visited)
        return False
    
    def __repr__(self) -> str:
        return f"PathExplorer({self._path}, {self._visited})"
    
    def __len__(self) -> int:
        return len(self._path)

In [141]:
def cycle_aux(
    grafo: WeightedDirectedGraph, explorer: PathExplorer, target: str
) -> tuple[list, str]:
    current_node = explorer.path[-1]
    if current_node == target:
        return explorer.path, "cycle"
    
    children = grafo.children(current_node)
    unexplored_nodes = children - explorer.visited
    if not unexplored_nodes:
        return [], "dead explorer"
    
    updated_visited = explorer.visited | {current_node}
    new_explorers = [
        PathExplorer(explorer.path + [node], updated_visited)
        for node in unexplored_nodes
    ]
    return new_explorers, "continue"

In [142]:
def find_cycles(grafo: WeightedDirectedGraph, start: str) -> list[list[str]]:
    first_children = grafo.children(start)
    if not first_children:
        return []

    explorers: list[PathExplorer] = [
        PathExplorer([start, child]) for child in first_children
    ]
    cycles: list[list[str]] = []

    while explorers: 
        result, state = cycle_aux(grafo, explorers.pop(0), start)
        if state == "cycle":
            cycles.append(result)
        elif state == "dead explorer":
            pass
        else: 
            new_explorers = list(set(explorers) - set(result))
            explorers.extend(result)

    return cycles

In [143]:
find_cycles(grafo, "A")

[['A', 'B', 'A'],
 ['A', 'B', 'C', 'A'],
 ['A', 'F', 'E', 'B', 'A'],
 ['A', 'B', 'C', 'D', 'A'],
 ['A', 'F', 'E', 'B', 'C', 'A'],
 ['A', 'F', 'E', 'B', 'C', 'D', 'A']]

In [144]:
find_cycles(grafo, "B")

[['B', 'A', 'B'],
 ['B', 'C', 'A', 'B'],
 ['B', 'A', 'F', 'E', 'B'],
 ['B', 'C', 'D', 'A', 'B'],
 ['B', 'C', 'A', 'F', 'E', 'B'],
 ['B', 'C', 'D', 'A', 'F', 'E', 'B']]

In [145]:
find_cycles(grafo, "C")

[['C', 'A', 'B', 'C'],
 ['C', 'D', 'A', 'B', 'C'],
 ['C', 'A', 'F', 'E', 'B', 'C'],
 ['C', 'D', 'A', 'F', 'E', 'B', 'C']]

In [146]:
find_cycles(grafo, "D")

[['D', 'A', 'B', 'C', 'D'], ['D', 'A', 'F', 'E', 'B', 'C', 'D']]

In [147]:
find_cycles(grafo, "E")

[['E', 'B', 'A', 'F', 'E'],
 ['E', 'B', 'C', 'A', 'F', 'E'],
 ['E', 'B', 'C', 'D', 'A', 'F', 'E']]

In [148]:
find_cycles(grafo, "F")

[['F', 'E', 'B', 'A', 'F'],
 ['F', 'E', 'B', 'C', 'A', 'F'],
 ['F', 'E', 'B', 'C', 'D', 'A', 'F']]

In [149]:
ciclo_1 = ['F', 'E', 'B', 'A', 'F']
ciclo_2 = ['E', 'B', 'A', 'F', 'E']

cleaned_1 = ciclo_1[:-1]
cleaned_2 = ciclo_2[:-1]

[cleaned_1[-i:] + cleaned_1[:-i] for i in range(len(cleaned_1))]

[['F', 'E', 'B', 'A'],
 ['A', 'F', 'E', 'B'],
 ['B', 'A', 'F', 'E'],
 ['E', 'B', 'A', 'F']]

In [181]:
from functools import cached_property

def _cycle_representations(cycle: list[str]) -> list[list[str]]:
    return [cycle[-i:] + cycle[:-i] for i in range(len(cycle))]

def _canonic_representation(cycle: list[str]) -> list[str]:
    first = min(cycle)
    canonic_representation = [
        representation
        for representation in _cycle_representations(cycle)
        if representation[0] == first
    ][0]
    return canonic_representation


class Cycle:

    def __init__(self, cycle: list[str]) -> None: 
        if len(cycle) < 2:
            raise ValueError("Cycle must have at least two elements")
        if cycle[0] == cycle[-1]:
            raise ValueError("Cycle can not contain equal consecutive elements")
        if any(cycle[i] == cycle[i+1] for i in range(len(cycle) - 1)):
            raise ValueError("Cycle can not contain equal consecutive elements")

        self._cycle = cycle

    @cached_property
    def cycle(self) -> list[str]:
        return _canonic_representation(self._cycle)
    
    @cached_property
    def equivalent_representations(self) -> list[list[str]]:
        return _cycle_representations(self._cycle)
    
    def __len__(self) -> int:
        return len(self._cycle)
    
    def __hash__(self) -> int:
        return hash(tuple(self.cycle))
    
    def __eq__(self, other: object) -> bool:
        if isinstance(other, Cycle):
            if len(other) != len(self):
                return False
            return other.cycle in self.equivalent_representations
        return False
    
    def __repr__(self) -> str:
        return f"Cycle({self.cycle})"
    


In [182]:
a = Cycle(["A", "B", "C"])
b = Cycle(["B", "C", "A"])

In [183]:
{b, b, a}

{Cycle(['A', 'B', 'C'])}

In [184]:
{1, 1}

{1}

In [171]:
min(["B", "C", "A"])

'A'